This notebook explores using GAP habitat metrics to cache additional information.

In [1]:
#Import needed packages
import geopandas as gpd
import json
import requests
from IPython.display import display
import bispy
from joblib import Parallel, delayed
import jsonschema
import shapefile

gap = bispy.gap.Gap()
bis_utils = bispy.bis.Utils()

In [2]:
#The WLCI boundary was exported as SHP from the ScienceBase item: https://www.sciencebase.gov/catalog/item/4f4e486fe4b07f02db50cfb5 as file WLCI_Expansion.shp.
#A field labeled as STATEFP was added to the record in order to run with the GAP bispy package. The integer 56 was added to the STATEFP field to represent the fips code for the state of Wyoming within which the WLCI boundary falls within.
wlci_boundary=gpd.read_file('sources/wlci_boundary')

In [3]:
wlci_boundary

,PERIMETER,Comment,AreaSqKM,Acres,Hectares,STATEFP,geometry
0,1513.336373,Wyoming Landscape Conservation Initiative (WLC...,62203.692212,1.905229e+07,7.710190e+06,56,"POLYGON ((592168.6260653427 4812997.806712992,..."


In [4]:
# Open up the cached GAP records from Check GAP.ipynb
with open("cache/gap.json", "r") as f:
    gap_cache = json.loads(f.read())

In [5]:
# Tease out the unique GAP species that we were able to find in the entire set of searched names
gap_species = list(map(json.loads,set(map(json.dumps, [r for r in gap_cache if "GAP Species" in r.keys()]))))

In [6]:
gap_species

[{'processing_metadata': {'status': 'success',
   'date_processed': '2019-08-27T20:35:20.011534',
   'status_message': 'Exact Match',
   'api': "https://www.sciencebase.gov/catalog/items?parentId=527d0a83e4b0850ea0518326&format=json&fields=identifiers,files,webLinks,distributionLinks,dates&filter=itemIdentifier%3D{'key': 'Amphispiza belli'}"},
  'parameters': {'Scientific Name': 'Amphispiza belli'},
  'GAP Species': {'GAP Habitat Map Item': 'https://www.sciencebase.gov/catalog/item/58fa5693e4b0b7ea545253c9',
   'GAP Range Map Item': 'https://www.sciencebase.gov/catalog/item/59f5ec3ae4b063d5d307e52f',
   'GAP Habitat Map WMS': 'https://www.sciencebase.gov/geoserver/CONUS_HabMap_2001/wms?service=WMS&version=1.1.0&request=GetCapabilities',
   'GAP Modeling Database Parameters URL': 'https://www.sciencebase.gov/catalog/file/get/58fa5693e4b0b7ea545253c9?f=__disk__ed%2F7d%2Fee%2Fed7deec515fd463b5a1e188123ccb1ef658c76cd',
   'GAP ITIS Information URL': 'https://www.sciencebase.gov/catalog/fil

In [7]:
wlci_gap_metrics = Parallel(n_jobs=8)(delayed(gap.gap_metrics_species)(wlci_boundary, i["GAP Species"]["GAP_SpeciesCode"], i["GAP Species"]["Range Bounding Box"]) for i in gap_species)

In [8]:
len(wlci_gap_metrics)

67

In [9]:
# Cache the array of retrieved documents and return/display a random sample for verification
display(bis_utils.doc_cache("cache/wlci_gap_range.json", wlci_gap_metrics))

{'Doc Cache File': 'cache/wlci_gap_range.json',
 'Number of Documents in Cache': 67,
 'Document Number 12': {'GAP_SpeciesCode': 'bRTHAx',
  'State Metrics': [{'state_fipscode': '56',
    'state_name': 'Wyoming',
    'taxa': 'B',
    'sppcode': 'bRTHAx',
    'spp_comname': 'Red-tailed Hawk',
    'spp_sciname': 'Buteo jamaicensis',
    'gapstat1ac': 5026093.4262362085,
    'gapstat2ac': 1592628.1822454256,
    'gapstat3ac': 27081088.072754927,
    'gapstat4ac': 25376136.928759586,
    'gapstat12ac': 6618721.608481634,
    'gapstat123ac': 33699809.681236565,
    'totalac': 59075946.60999615,
    'gapstat1perc': 8.50785085073144,
    'gapstat2perc': 2.69589955580321,
    'gapstat3perc': 45.8411411526541,
    'gapstat4perc': 42.9551084408112,
    'gapstat12perc': 11.2037504065346,
    'gapstat123perc': 57.0448915591888,
    'gapstat12group': '10-17',
    'gapstat123group': '>50',
    'id': 23701}]}}

Here the 2018 US State boundaries from Census, the GAP range bounding box (generated from the GAP species data from the GAP range services) for each species is used to determine which species have ranges within a give US state.

In [10]:
#Bring in the 2018 US State boundaries from Census
us_states = gpd.read_file("https://www2.census.gov/geo/tiger/TIGER2018/STATE/tl_2018_us_state.zip")

In [ ]:
# Use joblib to run multiple requests for GAP range records in parallel via GAP information
gap_metrics = Parallel(n_jobs=8)(delayed(gap.gap_metrics_species)(us_states, i["GAP Species"]["GAP_SpeciesCode"], i["GAP Species"]["Range Bounding Box"]) for i in gap_species)

In [ ]:
gap_results=[]
for item in gap_metrics:
    for record in item['State Metrics']:
        state_fipscode=record['state_fipscode']
        state=record['state_name'] 
        taxa=record['taxa']
        common_name=record['spp_comname']
        scientific_name=record['spp_sciname']
        gap_results.append({'state_fipscode':state_fipscode,'state':state,'taxa':taxa,'common_name':common_name,'scientific_name':scientific_name})

In [ ]:
wyoming=[i for i in gap_results if i['state'] == 'Wyoming']

In [ ]:
len(wyoming)